In [6]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [7]:
# # Restart kernel to clear memory
# !kill -9 -1


In [8]:
df = pd.read_csv('/kaggle/input/bengali-news/xlsum_bengali.csv')

In [9]:
df.dropna()

,title,summary,article
0,দুর্গাপূজার সময়ে যেভাবে শোক পালন করেন 'মহিষাস...,হিন্দু বাঙালীরা যে সময়ে তাদের সবথেকে বড় উৎসব...,দুর্গাপুজায় মহিষাসুর বধ্যে মধ্য দিয়ে অশুভর ও...
1,রাশিয়ায় ক্ষমতার ২০ বছর যেভাবে কেটেছে ভ্লাদিম...,ভ্লাদিমির পুতিন তাঁর ক্ষমতায় থাকার ২০ বছর পূর...,গত ২০ বছরে তিনি রাশিয়ার প্রেসিডেন্ট এবং প্রধা...
2,সংসদ নির্বাচন: বরিশালে ছয়টি আসন কিন্তু সবার দ...,বাংলাদেশের দক্ষিণাঞ্চলীয় জেলা বরিশাল এখন তুমু...,বরিশাল সদরে চলছে নির্বাচনী প্রচার প্রচারণা। যদ...
3,সর্বকালের সর্বশ্রেষ্ঠ বাঙালি: বিবিসি বাংলার জর...,দু'হাজার চার সালে বিবিসি বাংলা একটি 'শ্রোতা জর...,রবীন্দ্রনাথ ঠাকুর রবীন্দ্রনাথ ঠাকুর বাঙালির কা...
4,৭ই মার্চের ভাষণ: ৫০ বছর আগে রেসকোর্স ময়দানে উ...,'ভাষণ শুরু আগে মাথার উপর দিয়ে বিমান আর হেলিকপ...,আর কুমিল্লা থেকে বাস ভাড়া করে অনেকের সাথে নিজ...
...,...,...,...
10121,"করোনাভাইরাস: মহামারির আশঙ্কা, বিশ্বজুড়ে আসতে ...",করোনাভাইরাসে আক্রান্ত হয়ে মৃত্যুর সংখ্যা ১৭০ ...,বিশ্ব স্বাস্থ্য সংস্থা বলছে মহামারী আকারে পুরো...
10122,বুনো হাতিটির পেছনে ভারত-বাংলাদেশের বনকর্মীরা,বাংলাদেশের জামালপুরের একটি চরে আটকে থাকা ভারতী...,জামালপুরের প্রত্যন্ত চরে আটকে রয়েছে ভারতীয় এ...
10123,সৌদি আরব নারীদের পুরুষ অভিভাবক ছাড়া একা বিদেশ...,এখন থেকে সৌদি আরবের নারীরা কোনো পুরুষ অভিভাবকে...,ভ্রমণের অধিকারের হিসেবে সৌদি আরবের নারীদেরকে প...
10124,দিল্লীতে কিশোরদের ইনস্টাগ্রাম গ্রুপে গণধর্ষণের...,ইনস্টাগ্রাম গ্রুপ চ্যাটে অপ্রাপ্তবয়স্ক মেয়েদ...,মেয়েদের নিয়ে যৌন হয়রানিমূলক মন্তব্য করায় গ...


In [10]:
df = df.dropna(subset=['article', 'summary'])

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.30, shuffle=True)
df_val, df_test = train_test_split(df_test, test_size=0.65,shuffle=True)

In [12]:
# from datasets import Dataset
# ds_train = Dataset.from_pandas(df_train)
# ds_test = Dataset.from_pandas(df_test)

In [13]:
df_train['summary'][50]

'তিরিশ বছর আগে ১৯৯০ সালের ৬ই ডিসেম্বর এক তীব্র গণআন্দোলনের মুখে বাংলাদেশে জেনারেল এরশাদের নয় বছরের শাসনের অবসান ঘটে। কিন্তু তারও তিন বছর আগে আরেকটি গণআন্দোলন তিনি নিষ্ঠুরভাবে দমন করেছিলেন। সেই আন্দোলনের সময় গণতন্ত্রের দাবিতে বুকে-পিঠে শ্লোগান লিখে রাস্তায় নামা এক তরুণ নূর হোসেন পুলিশের গুলিতে নিহত হন। মৃত্যুর আগে তোলা তার একটি ছবি বাংলাদেশের গণতান্ত্রিক আন্দোলনের প্রতীক হয়ে উঠে এবং তিন বছর পরের গণআন্দোলনকে অনুপ্রাণিত করে। ১৯৮৭ সালের ১০ নভেম্বর নূর হোসেনের বুলেটবিদ্ধ দেহ যে কারাকক্ষে ফেলে রাখা হয়, তার পাশের কক্ষে বন্দী ছিলেন বিবিসি বাংলার মোয়াজ্জেম হোসেন। তেত্রিশ বছর পর নূর হোসেনের সেই অভিনব প্রতিবাদের নেপথ্যে কাহিনী জানার চেষ্টা করেছেন তিনি।'

In [14]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os


# Specify the model name for mT5
model_name = "google/mt5-base"  # You can also use 'mt5-small', 'mt5-large', etc.

# Load the mT5 model
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# Load the mT5 tokenizer
tokenizer = MT5Tokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
model_name

'google/mt5-base'

In [16]:
from torch.utils.data import Dataset, DataLoader

class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        # Drop rows with NaN values in article or summary columns
        data = data.dropna(subset=['article', 'summary'])

        # Tokenize input text and labels (without normalization)
        self.input_text = data['article'].tolist()
        self.labels = data['summary'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        # Tokenize the input and labels
        inputs = self.tokenizer(self.input_text[idx], 
                                max_length=self.max_length, 
                                padding='max_length', 
                                truncation=True, 
                                return_tensors="pt")

        labels = self.tokenizer(self.labels[idx], 
                                max_length=self.max_length, 
                                padding='max_length', 
                                truncation=True, 
                                return_tensors="pt")

        # Return input and labels for the sample
        return {
            'input_ids': inputs['input_ids'].squeeze(), 
            'attention_mask': inputs['attention_mask'].squeeze(), 
            'labels': labels['input_ids'].squeeze()
        }


In [17]:
import torch
from transformers import DataCollatorForSeq2Seq
from torch.nn.utils.rnn import pad_sequence

class MyDataCollatorForSeq2Seq(DataCollatorForSeq2Seq):
    def __call__(self, features):
        batch = {}

        # Handle input_ids and attention_mask
        batch["input_ids"] = torch.stack([feature["input_ids"] for feature in features])
        batch["attention_mask"] = torch.stack([feature["attention_mask"] for feature in features])

        # Check if the labels are PyTorch tensors
        if isinstance(features[0]["labels"], torch.Tensor):
            # Ensure padding for labels is handled correctly
            batch["labels"] = torch.stack([feature["labels"] for feature in features])
        else:
            # Convert list of lists (if not tensor) to tensor and pad
            batch["labels"] = torch.tensor([feature["labels"] for feature in features], dtype=torch.long)

        # Make sure labels are padded if necessary (for variable-length sequences)
        if self.label_pad_token_id is not None:
            batch["labels"] = pad_sequence([feature["labels"] for feature in features], 
                                           batch_first=True, 
                                           padding_value=self.label_pad_token_id)

        return batch


In [18]:
# Create train, test, and validation datasets
train_dataset = Seq2SeqDataset(df_train, tokenizer)
test_dataset = Seq2SeqDataset(df_test, tokenizer)
validation_dataset = Seq2SeqDataset(df_val, tokenizer)

# Create DataLoader for each dataset
train_dataloader = DataLoader(train_dataset, 
                              batch_size=16, 
                              shuffle=True, 
                              collate_fn=MyDataCollatorForSeq2Seq(tokenizer))

test_dataloader = DataLoader(test_dataset, 
                             batch_size=16, 
                             collate_fn=MyDataCollatorForSeq2Seq(tokenizer))

validation_dataloader = DataLoader(validation_dataset, 
                                   batch_size=16, 
                                   collate_fn=MyDataCollatorForSeq2Seq(tokenizer))


In [19]:
# Move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
device

device(type='cuda')

In [21]:
# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    eps=1e-8,
    weight_decay=0.01,
)

In [27]:
# Define the TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    num_train_epochs=6,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_total_limit=1,
    save_steps=5000,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=False,
    lr_scheduler_type="cosine_with_restarts",
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='/kaggle/working/',
    logging_steps=200,
    
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
# a7eadb1b11af8e041a6e1104b836b061cb0ff681


In [28]:
data_collator = MyDataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,  # Padding to the model's max length
    max_length=128,  # Use the same max length as during tokenization
    label_pad_token_id=tokenizer.pad_token_id,
)


In [30]:
from transformers.trainer import Trainer

class CustomTrainer(Trainer):
    def _save(self, output_dir=None, state_dict=None):
        # Ensure all tensors are contiguous before saving
        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()
        
        # Continue with normal save operation
        super()._save(output_dir, state_dict)

# Use CustomTrainer instead of Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),
)


In [31]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
0,7.364100,2.823433
2,1.558500,1.385688
4,1.310800,1.305233
5,1.263300,1.304631


TrainOutput(global_step=1326, training_loss=2.4029517655638726, metrics={'train_runtime': 3379.6197, 'train_samples_per_second': 12.584, 'train_steps_per_second': 0.392, 'total_flos': 1.2719481365200896e+16, 'train_loss': 2.4029517655638726, 'epoch': 5.986455981941309})

In [32]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f452fc5be6ac52144971ee24d5c3e4d1306a1aa561f746f90917aca0ec8063c2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.3 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Evaluate ROUGE scores

In [35]:
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Smoothing function for BLEU
smoothing_function = SmoothingFunction().method4

# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch

# Initialize lists to store generated summaries and references
generated_summaries = []
references = []

# Generate summaries for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']
    labels = batch['labels']

    # Generate summaries
    with torch.no_grad():
        summary_ids = model.generate(
            input_text,
            num_beams=6,
            length_penalty=1.0,
            early_stopping=True
        )

    # Move the summary_ids to CPU to decode
    summary_ids = summary_ids.to('cpu')

    # Decode generated summaries and labels
    generated_summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    reference_summary = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Extend lists with generated summaries and references
    generated_summaries.extend(generated_summary)
    references.extend(reference_summary)

# Filter out empty predictions and references
filtered_generated_summaries = []
filtered_references = []

for pred, ref in zip(generated_summaries, references):
    if pred.strip() and ref.strip():  # Check if both are non-empty
        filtered_generated_summaries.append(pred)
        filtered_references.append(ref)

# Ensure the filtered lists are populated before calculating BLEU and ROUGE
print(f"Filtered Generated Summaries Count: {len(filtered_generated_summaries)}")
print(f"Filtered References Count: {len(filtered_references)}")

# BLEU score calculation
def calculate_bleu(filtered_generated_summaries, filtered_references, tokenizer):
    bleu_scores = []
    for pred, ref in zip(filtered_generated_summaries, filtered_references):
        pred_tokens = tokenizer.tokenize(pred)
        ref_tokens = tokenizer.tokenize(ref)
        if pred_tokens and ref_tokens:  # Ensure both are non-empty
            score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothing_function)
            bleu_scores.append(score)

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    return avg_bleu_score

# Calculate BLEU
bleu_score = calculate_bleu(filtered_generated_summaries, filtered_references, tokenizer)
print(f"BLEU Score: {bleu_score}")



/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Filtered Generated Summaries Count: 1975
Filtered References Count: 1975
BLEU Score: 0.04643894758914907


In [38]:
from rouge_score import rouge_scorer
from unidecode import unidecode

# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
# Define a function to preprocess and tokenize Bengali text
def preprocess_text(text):
    text = unidecode(text)
    tokens = text.split()
    return ' '.join(tokens)

# Create lists to store individual scores
rouge1_f1_scores = []
rouge1_precision_scores = []
rouge1_recall_scores = []
rouge2_f1_scores = []
rouge2_precision_scores = []
rouge2_recall_scores = []
rougeL_f1_scores = []
rougeL_precision_scores = []
rougeL_recall_scores = []

for ref, pred in zip(references, generated_summaries):
    candidate = preprocess_text(pred)
    reference = preprocess_text(' '.join(ref))
    scores = scorer.score(reference, candidate)

    rouge1_f1_scores.append(scores['rouge1'].fmeasure)
    rouge1_precision_scores.append(scores['rouge1'].precision)
    rouge1_recall_scores.append(scores['rouge1'].recall)
    rouge2_f1_scores.append(scores['rouge2'].fmeasure)
    rouge2_precision_scores.append(scores['rouge2'].precision)
    rouge2_recall_scores.append(scores['rouge2'].recall)
    rougeL_f1_scores.append(scores['rougeL'].fmeasure)
    rougeL_precision_scores.append(scores['rougeL'].precision)
    rougeL_recall_scores.append(scores['rougeL'].recall)

# Calculate the average scores
avg_rouge1_f1 = sum(rouge1_f1_scores) / len(rouge1_f1_scores)
avg_rouge1_precision = sum(rouge1_precision_scores) / len(rouge1_precision_scores)
avg_rouge1_recall = sum(rouge1_recall_scores) / len(rouge1_recall_scores)
avg_rouge2_f1 = sum(rouge2_f1_scores) / len(rouge2_f1_scores)
avg_rouge2_precision = sum(rouge2_precision_scores) / len(rouge2_precision_scores)
avg_rouge2_recall = sum(rouge2_recall_scores) / len(rouge2_recall_scores)
avg_rougeL_f1 = sum(rougeL_f1_scores) / len(rougeL_f1_scores)
avg_rougeL_precision = sum(rougeL_precision_scores) / len(rougeL_precision_scores)
avg_rougeL_recall = sum(rougeL_recall_scores) / len(rougeL_recall_scores)

In [39]:
# Print the average scores
print("Average Rouge-1 F1 Score:", avg_rouge1_f1)
print("Average Rouge-1 Precision:", avg_rouge1_precision)
print("Average Rouge-1 Recall:", avg_rouge1_recall)

print("Average Rouge-2 F1 Score:", avg_rouge2_f1)
print("Average Rouge-2 Precision:", avg_rouge2_precision)
print("Average Rouge-2 Recall:", avg_rouge2_recall)

print("Average Rouge-L F1 Score:", avg_rougeL_f1)
print("Average Rouge-L Precision:", avg_rougeL_precision)
print("Average Rouge-L Recall:", avg_rougeL_recall)

Average Rouge-1 F1 Score: 0.0177801176599677
Average Rouge-1 Precision: 0.10865618207390616
Average Rouge-1 Recall: 0.009833039682601427
Average Rouge-2 F1 Score: 0.0
Average Rouge-2 Precision: 0.0
Average Rouge-2 Recall: 0.0
Average Rouge-L F1 Score: 0.0177801176599677
Average Rouge-L Precision: 0.10865618207390616
Average Rouge-L Recall: 0.009833039682601427


In [ ]:
generated_summaries

In [ ]:
# from torch.utils.data import DataLoader
# import torch

# # Assuming tokenized_df_test has been defined and is in the correct format
# # Create DataLoader for the test set
# sample_dataloader = DataLoader(
#     tokenized_df_test,  # use the tokenized version
#     collate_fn=data_collator,  # Ensure this collator is defined to handle padding
#     batch_size=2
# )

# predictions = []
# references = []

# # Process the entire test set
# for batch in sample_dataloader:
#     with torch.no_grad():
#         # Ensure input_ids and labels are moved to the correct device (e.g., GPU)
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         # Generate predictions
#         preds = model.generate(
#             input_ids,
#             num_beams=2,
#             num_return_sequences=1,
#             no_repeat_ngram_size=1,
#             remove_invalid_values=True,
#             max_length=128
#         )
    
#     # Decode the predictions and labels
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Store the decoded predictions and labels
#     predictions.extend(decoded_preds)
#     references.extend(decoded_labels)

# # Prepare the references for metric evaluation (references need to be wrapped in lists)
# references = [[ref] for ref in references]

# # Compute the metrics using already decoded text (BLEU/ROUGE)
# metrics = metrics_func(predictions=predictions, references=references)
# print(metrics)


In [ ]:
# def metrics_func(eval_arg):
#     preds, labels = eval_arg

#     # Ensure the Bengali punctuation logic is applied
#     text_preds = [(p if p.endswith(("!", "?", "।")) else p + "।") for p in preds]
#     text_labels = [(l if l.endswith(("!", "?", "।")) else l + "।") for l in labels]

#     # Tokenize sentences in Bengali
#     sent_tokenizer_bn = RegexpTokenizer(u'[^!!??।]*[!!??।]')
#     text_preds = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(p))) for p in text_preds]
#     text_labels = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(l))) for l in text_labels]

#     # Compute the ROUGE score directly from the text
#     return rouge_metric.compute(predictions=text_preds, references=text_labels)


In [ ]:
# from datasets import load_metric

# # Load ROUGE and/or BLEU metrics
# rouge = load_metric("rouge", trust_remote_code=True)
# bleu = load_metric("bleu")

# def metrics_func(predictions, references):
#     rouge_result = rouge.compute(predictions=predictions, references=references)
#     bleu_result = bleu.compute(predictions=predictions, references=references)
#     return {"rouge": rouge_result, "bleu": bleu_result}


In [ ]:
# # Compute the metrics using already decoded text (BLEU/ROUGE)
# metrics = metrics_func(predictions=predictions, references=references)
# print(metrics)